# Problem 5.1

In [2]:
import collections
import warnings

# The workhorses
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy.optimize
import numdifftools as ndt

# Plotting modules
import matplotlib.pyplot as plt

# MCMC utilities
import emcee

# Numba for JITting log posteriors
import numba

# BE/Bi 103 utilities
import sys
sys.path.append('C:/Users/Elena/Desktop/Caltech/BEBi 103/bebi103_utils')
import bebi103

# This is to enable inline displays for the purposes of the tutorial
%matplotlib inline

# This enables SVG graphics inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Seaborn makes plots look nice
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.linewidth': 2.5})
sns.set_style('darkgrid', {'axes.facecolor': '(0.875, 0.875, 0.9)'})

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ImportError: No module named 'bebi103'

# Part A

In [6]:
# Load data
df = pd.read_csv('./data/mean_rest_bouts.csv', comment='#')

# Pull out wild type and mutant and take NaNs to be zero
df = df[df['genotype'].isin(['wt', 'mut'])].fillna(0)
df.head()

,fish,genotype,mean_rest_bout_length
2,FISH11,mut,2.255556
3,FISH12,mut,1.529412
4,FISH13,mut,2.373626
5,FISH14,wt,2.352941
7,FISH18,wt,2.111111


In [7]:
# Make strip plot
sns.stripplot(x='genotype', y='mean_rest_bout_length', data=df, jitter=True)
# Label axes 
plt.ylabel('mean rest bout length (min)')
plt.margins(y=0.02)

NameError: name 'sns' is not defined

In [8]:
#separate the two datasets in to wild type and mutant
wt_fish=df[df["genotype"]=="wt"]
mut_fish=df[df["genotype"]=="mut"]

Sample mean is the best estimate of $\mu$. Unbiased estimator of variance is:
\begin{align}
\frac{n}{n-1}&s^2
\end{align}

In [9]:
#find means for wt and mut
mu_wt = wt_fish.mean()
mu_mut = mut_fish.mean()

#find unbiased estimator of variance for wt and mut
var_wt = wt_fish.var(ddof=1)
var_mut = mut_fish.var(ddof=1)

print("""The best estimates for µ and σ are:
            µ           σ
         -----------------
WT       {0:.4f}     {2:.4f}
Mutant   {1:.4f}     {3:.4f}
""".format(float(mu_wt), float(mu_mut), 
           float(np.sqrt(var_wt)), float(np.sqrt(var_mut))))

The best estimates for µ and σ are:
            µ           σ
         -----------------
WT       2.2094     0.5229
Mutant   1.7271     0.8051



# Part B

Frequentist estimate of the difference of the means:
\begin{align}
\delta \equiv \mu_{wt} - \mu_{mut}
\end{align}

In [10]:
def gaussian_draw(WTmu, WTsigma, MUTmu, MUTsigma, WTreps, MUTreps, trials):
    """
    
    """
    difference_mean_holder=np.empty(trials)
    
    for i in range(trials):
        wt_rep_mean = np.random.normal(WTmu, WTsigma, WTreps).mean()
        mut_rep_mean = np.random.normal(MUTmu, MUTsigma, MUTreps).mean()
        difference_mean_holder[i]=wt_rep_mean - mut_rep_mean
        
    return difference_mean_holder


In [11]:
difference_out = gaussian_draw(mu_wt, np.sqrt(var_wt),mu_mut, 
                               np.sqrt(var_mut), len(wt_fish), 
                               len(mut_fish), 10000)

In [12]:
plt.hist(difference_out, histtype="step", bins=100, color="blue", lw=2)
conf_interval = np.percentile(difference_out, (2.5,97.5))
plt.xlabel("δ")
plt.ylabel("P(δ)")
plt.title("δ = μ(wt)−μ(mut)")

In [13]:
print ("The 95% confidence interval is", conf_interval, "minutes")
print("""The final result is:
      µ(wt)-µ(mut) = {0:.4f} ± {1:.4f}"""
      .format(float(mu_wt-mu_mut), float(1.96*np.std(difference_out))), "minutes")

The 95% confidence interval is [ 0.06717094  0.89611682] minutes
The final result is:
      µ(wt)-µ(mut) = 0.4823 ± 0.4153 minutes


# Part C

In [14]:
#this code has been modified from previous HW solutions.

# Define log of the posterior
def log_post(p, x_wt, x_mut):
    """
    Returns the log of the posterior consisting of the product of Gaussians.
    p[0] = mu_wt
    p[1] = mu_mut
    p[2] = sigma_wt
    p[3] = sigma_mut
    """
    # Unpack parameters
    mu_wt, mu_mut, sigma_wt, sigma_mut = p
    
    # Make sure we have everything in the right range
    if (sigma_wt < 0) or (sigma_mut < 0):
        return -np.inf

    # Compute separate parts of posterior from each sample
    log_post_wt = st.norm.logpdf(x_wt, mu_wt, sigma_wt).sum() - np.log(sigma_wt)
    
    log_post_mut = st.norm.logpdf(x_mut, mu_mut, sigma_mut).sum() - np.log(sigma_mut)

    # Add them up to return
    return log_post_wt + log_post_mut

def negative_log_post(p, x_wt,x_mut):
    return -log_post(p,x_wt,x_mut)

#MAP
wt=wt_fish[("mean_rest_bout_length")]
mut=mut_fish[("mean_rest_bout_length")]
args = (wt,mut)

p0 = np.array([ 2, 1.5, 0.5, 0.8])

hes_fun = ndt.Hessian(log_post)
res = scipy.optimize.minimize(negative_log_post, p0, args=args)
hes = hes_fun(res.x,wt,mut)
cov1 = -np.linalg.inv(hes)

bayesian_estimate_diff = res.x[0]-res.x[1]
credible_interval = 1.96 * np.sqrt(np.sqrt(cov1[0,0])**2 + np.sqrt(cov1[1,1])**2)

print("The bayesian estimate of the difference of the mean is", 
      bayesian_estimate_diff, "±", credible_interval)

The bayesian estimate of the difference of the mean is 0.482335452396 ± 0.397809846791


# Part D

First we make an array of the mean rest bout lengths for our set of wild type fish and mutant fish.

In [15]:
# we want to make an array of the mean rest bout lengths
wt = wt_fish['mean_rest_bout_length'].values

mut = mut_fish['mean_rest_bout_length'].values

Now we want to test our null hypothesis. $H_0$ is that the wild type and mutant are drawn from the same Gaussian distribution, more specifically a student-t distribution. In other words that the means are the same $\mu_{wt} = \mu_{mut}$. 

We can use a T-statistic as our measurable statistic. 
\begin{align}
T = \frac {\bar{x}_1 - \bar{x}_2 - n_1 + n_2} {S_D \sqrt{n^{-1}_1 + n^{-1}_2}}
\end{align}
Where $S_D$ is defined as:
\begin{align}
S_D = \frac {(n_1 - 1)S_1^2 + (n_2 - 1)S_2^2} {n_1 + n_2 -2}
\end{align}
And S_1 and S_2 are
\begin{align}
S = \frac {1}{n - 1} \sum_{i \in D}{(x_i - \bar{x})^2}
\end{align}
for their respective values.

We want to combine all of our measurements and then redistribute them to a group of 17 and group of 22. 

In [16]:
# our null hypothesis is that the mutant and the wild type are drawn from the same
# Gaussian distribution 

# We first combine all of the measurements and then distrbute them to a set of 17 and 
# a set of 22

def final_p_value(w, m, trials, greater_or_less='greater', return_abs=False):
    
    def distribute_samples(w, m, some_func, trials, args=()):
        '''
        combine our measurements for rest bout and redistribute them to two groups
        '''
    
        # first we create an empty array to store our samples
        samples = np.empty(trials)
    
        # concatenate all the measurements
        measurements = np.concatenate((w, m))
    
        # create sets for all trials 
        for i in range(trials):
            measurements = np.random.permutation(measurements)
            samples[i] = some_func(measurements[:len(w)], measurements[len(w):], *args)
     
        return samples


    # now I want to define the p-value
    def p_value(samples, actual, greater_or_less='greater'):
        '''
        compute the p value (the probability our measurement is 
        greather than or less than the actual value
        '''
    
        if greater_or_less == 'greater':
            return np.sum(samples >= actual) / len(samples)
        else:
            return np.sum(samples <= actual) / len(samples)

    # now we can run our test and see if we will get the actual difference in our means
    # this will be my some_func in my distribute_samples function
    # now we can run our a Welch's t-test
    # The T statistic is written out in tutorial 6b
    def T_stat(w, m, return_abs=False):
        '''
        Calculate the absolute difference of the means
        '''
        w_s = w.var(ddof=1)
        m_s = m.var(ddof=1)
    
        sd = ((len(w)-1) * w_s**2 + (len(m) - 1) * m_s**2) / (len(w) + len(m) - 2)
    
        diff = w.mean() - m.mean() - np.sqrt(w.var()) + np.sqrt(m.var()) 
        pool_variance = sd * np.sqrt(1/(len(w)) + 1/(len(m)))
            
        if return_abs:
            return np.abs(diff) / pool_variance   
        return diff / pool_variance


    # let's retrieve all our random sample sets of two
    samples = distribute_samples(w, m, T_stat, trials, args = (True,))

    # calculate p value
    p_value_student_t = p_value(samples, T_stat(w, m))

    return p_value_student_t

p_value = final_p_value(wt, mut, 100000)
print("Welch's t test p value =", p_value)

Welch's t test p value = 0.02204


# Part E

In [18]:
# Define log of the posterior for H_0 
def log_post_H0(p, x):
    """
    Returns the log of the posterior for a Gaussian.
    p[0] = mu
    p[1] = sigma
    """
    # Unpack parameters
    mu, sigma = p
    
    # Make sure we have everything in the right range
    if (sigma < 0):
        return -np.inf

    # Add them up to return
    return st.norm.logpdf(x, mu, sigma).sum() - np.log(sigma)

In [19]:
# Instantiate Hessian for log posterior for H_0
hes_fun = ndt.Hessian(log_post_H0)

# Compute Hessian at MAP for H_0
hes = hes_fun(np.asarray([df['mean_rest_bout_length'].mean(), 
                          df['mean_rest_bout_length'].std()]), df['mean_rest_bout_length'])

# Calculate covariance matrix for H_0
cov0 = -np.linalg.inv(hes)

In [20]:
# Calculate the log of the good fit ratio
log_good_fit_ratio = \
st.norm.logpdf(df['mean_rest_bout_length'], df['mean_rest_bout_length'].mean(), df['mean_rest_bout_length'].std()).sum() \
- st.norm.logpdf(wt, res.x[0], res.x[2]).sum() \
- st.norm.logpdf(mut, res.x[1], res.x[3]).sum()

# Caclulate the log of the Occam factor
log_occam = np.log(res.x[2]) + np.log(res.x[3]) + np.log(2 * np.pi) + np.log(np.linalg.det(cov0))/2 \
- np.log(df['mean_rest_bout_length'].std()) - 2 * np.log(2 * np.pi) - np.log(np.linalg.det(cov1))/2

# Calculate and print the odds ratio
print("""Odds ratio = {0:.4f}""".format(np.exp(log_good_fit_ratio + log_occam)))

Odds ratio = 0.0830


# Part F

**d)**

In [ ]:
p_val = final_p_value(wt_set, mut_set, 100000)
print('p value = ',p_val)